In [1]:
import os
from pathlib import Path

path = Path('data/bodyfat_dataset.csv')


In [2]:
import torch, numpy as np, pandas as pd

df = pd.read_csv(path)

df

,title,url,meanPrediction,medianPrediction,bfPredictions,image_1,image_2,image_3,image_4,image_5
0,Leanest ive ever been. Never seen veins like t...,https://www.reddit.com/gallery/1b6k5jh,8.00,8.0,[8],https://preview.redd.it/leanest-ive-ever-been-...,https://preview.redd.it/leanest-ive-ever-been-...,https://preview.redd.it/leanest-ive-ever-been-...,https://preview.redd.it/leanest-ive-ever-been-...,NaN
1,Let me know. 78kg :),https://i.redd.it/4occeq9wdd2c1.jpg,9.80,10.0,"[9, 10, 10, 9, 11]",https://i.redd.it/4occeq9wdd2c1.jpg,NaN,NaN,NaN,NaN
2,What is my bf% I believe it’s around 13-14%,https://i.redd.it/lvfmowq0zhh91.jpg,14.33,13.5,"[13, 19, 14, 14, 13, 13]",https://i.redd.it/lvfmowq0zhh91.jpg,NaN,NaN,NaN,NaN
3,"25F | 4'11"" | 107 lbs",https://www.reddit.com/gallery/1e30z2f,23.33,24.0,"[20, 26, 24]",https://preview.redd.it/25f-411-107-lbs-v0-4lm...,https://preview.redd.it/25f-411-107-lbs-v0-huc...,NaN,NaN,NaN
4,Bodyfat?,https://www.reddit.com/gallery/1ktwao9,17.80,18.0,"[18, 16, 18, 15, 22]",https://preview.redd.it/8ekhv0d4xl2f1.jpg?widt...,https://preview.redd.it/rwkt50d4xl2f1.jpg?widt...,https://preview.redd.it/vau412d4xl2f1.jpg?widt...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
801,[GMBF] 6'1 220LBS Any estimates appreciated!,https://i.redd.it/z543jfquva731.jpg,12.00,12.0,[12],https://i.redd.it/z543jfquva731.jpg,NaN,NaN,NaN,NaN
802,"[GMBF] (M/22/6'2""/195lbs)",https://i.imgur.com/PY44dK5.jpg,10.50,10.5,"[11, 9, 10, 12]",https://i.imgur.com/PY44dK5.jpg,NaN,NaN,NaN,NaN
803,[GMBF] M/26/6'1/191lbs - down from 245lbs,https://i.imgur.com/pWGT7nV.jpg,15.00,15.0,[15],https://i.imgur.com/pWGT7nV.jpg,NaN,NaN,NaN,NaN
804,[GMBF] (M/26/5’10”/153lbs to 168lbs) What woul...,https://i.redd.it/lc2q2svumrt11.jpg,12.00,12.0,"[10, 14]",https://i.redd.it/lc2q2svumrt11.jpg,NaN,NaN,NaN,NaN


In [3]:
import requests
from tqdm import tqdm

def download_all_images(df, output_dir="raw_images"):
    os.makedirs(output_dir, exist_ok=True)
    image_cols = [f"image_{i}" for i in range(1, 6)]
    
    seen_urls = set()
    image_count = 0

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        for col in image_cols:
            url = row.get(col)
            if isinstance(url, str) and url.startswith("http") and url not in seen_urls:
                try:
                    response = requests.get(url, timeout=10)
                    if response.status_code == 200:
                        file_ext = url.split('.')[-1].split('?')[0]
                        file_name = f"{idx}_{col}.{file_ext}"
                        file_path = os.path.join(output_dir, file_name)
                        with open(file_path, "wb") as f:
                            f.write(response.content)
                        seen_urls.add(url)
                        image_count += 1
                except Exception as e:
                    print(f"Error downloading {url}: {e}")

    print(f"\nDownloaded {image_count} unique images to '{output_dir}/'")

In [4]:
def create_regression_csv(df, output_csv="image_labels.csv", label_col="meanPrediction", image_prefix="image_", output_dir="images"):
    # Ensure column names are stripped of whitespace
    df.columns = df.columns.str.strip()
    
    image_cols = [col for col in df.columns if col.startswith(image_prefix)]
    records = []

    for idx, row in df.iterrows():
        label = row[label_col]
        for col in image_cols:
            url = row.get(col)
            if isinstance(url, str) and url.startswith("http"):
                ext = url.split('.')[-1].split('?')[0].lower()
                ext = ext if ext in ['jpg', 'jpeg', 'png', 'webp'] else 'jpg'
                filename = f"{idx}_{col}.{ext}"
                records.append({"filename": filename, "target": label})
    
    df_out = pd.DataFrame(records)
    df_out.to_csv(output_csv, index=False)
    print(f"Created {output_csv} with {len(df_out)} labeled images")
    return df_out

In [5]:
download_all_images(df)  

df_labels = create_regression_csv(df) 
df_labels.head()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 806/806 [03:29<00:00,  3.84it/s]


Downloaded 1579 unique images to 'raw_images/'
Created image_labels.csv with 1597 labeled images


,filename,target
0,0_image_1.jpg,8.0
1,0_image_2.jpg,8.0
2,0_image_3.jpg,8.0
3,0_image_4.jpg,8.0
4,1_image_1.jpg,9.8


In [6]:
from fastai.vision.all import *

failed = verify_images(get_image_files(Path('raw_images')))
failed.map(Path.unlink)
len(failed)

0

In [7]:
def crop_upper_body(image_path):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Failed to read image {image_path}")
        return None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    if not results.pose_landmarks:
        print(f"No pose detected in {image_path}")
        return None

    try:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]

        h, w, _ = img.shape
        xs = [left_shoulder.x * w, right_shoulder.x * w, left_hip.x * w, right_hip.x * w]
        ys = [left_shoulder.y * h, right_shoulder.y * h, left_hip.y * h, right_hip.y * h]

        xmin, xmax = int(min(xs)), int(max(xs))
        ymin, ymax = int(min(ys)), int(max(ys))

        # Add padding around landmarks
        pad_x = 20
        pad_y = 20
        xmin = max(xmin - pad_x, 0)
        xmax = min(xmax + pad_x, w)
        ymin = max(ymin - pad_y, 0)
        ymax = min(ymax + pad_y, h)

        # Final adjustment: widen the crop a little more (e.g., 10% of width)
        box_width = xmax - xmin
        expand = int(box_width * 0.1)  # Expand 10% on each side
        xmin = max(xmin - expand, 0)
        xmax = min(xmax + expand, w)

        cropped = img[ymin:ymax, xmin:xmax]
        cropped_rgb = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
        return Image.fromarray(cropped_rgb)

    except Exception as e:
        print(f"Error cropping {image_path}: {e}")
        return None


ModuleNotFoundError: No module named 'mediapipe'